# Генератор шагов алгоритма Якоби для решения уравнения Пуассона

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams.update({'font.size': 16})
plt.rcParams['figure.figsize'] = (10,5)

In [ ]:
def solve_poisson_to(file, N, T1, T2, f, tol = 1e-3, max_it = 1e+4):
    """
    file --- файл, куда сохранять итерации
    N-1 --- количество внутренних узлов сетки
    T1  --- температура на левой и нижней границе
    T2  --- темепература на середине верхней и правой границ
    f --- массив (N-1, N-1), правая часть уравнения Пуассона
    tol --- относительная неувязка по целевой функции
    max_i --- макимальное количество операций
    """
    h = 1.0/N
    u = np.zeros((N+1, N+1)) # initialize 2-d solution array
    u_new = np.zeros((N+1, N+1))
    x = np.arange(N+1) * h # x = i * h, i = 0,...,N
    y = np.arange(N+1) * h # y = j * h, j = 0,...,N
    # set boundary conditions
    history = np.zeros((N + 1, N + 1, max_it))
    u_new[:, 0] = T1 
    u_new[0, :] = T1
    u_new[:, N] = np.where(np.logical_or(x<0.4, x>0.6), T1, T2)         
    u_new[N, :] = np.where(np.logical_or(y<0.2, y>0.8), T1, T2) 
    it = 0
    x_index = np.repeat(np.arange(1, N)[:, np.newaxis], N-1, axis=1)
    y_index = np.repeat(np.arange(1, N)[np.newaxis, :], N-1, axis=0)
    while True:
        u = np.copy(u_new)
        u_new[x_index, y_index] = 0.25 * (u[x_index + 1, y_index] + 
                                        u[x_index - 1, y_index] + 
                                        u[x_index, y_index + 1] + 
                                        u[x_index, y_index - 1] - h ** 2 * f)
        delta = np.max(np.abs(u_new - u)) / np.max(np.abs(u_new))
        history[:, :, it] = u_new
        it = it + 1
        if ((delta < tol) or (it > max_it)):
            break
    np.save(file, history)
    return u_new.T, x, y, it, delta

In [ ]:
T1 = 300
T2 = 500
u, x, y, it, delta = solve_poisson(N = 40, T1 = T1, T2 = T2, f=0, tol = 1e-6, max_it=1e+4)
T_c = u[u.shape[0]//2, u.shape[0]//2]
print('it = {0:d}, delta = {1:5.2e}, u(0.5, 0.5) = {2:5.2f}'.format(it, delta, T_c))
#
# Draw figures
#
X, Y = np.meshgrid(x, y)
fig = plt.figure(figsize = (16,8))
ax1 = fig.add_subplot(1, 2, 1)
cf = ax1.contourf(X, Y, u, 100, cmap = 'jet')
ax1.set_title('T1 = {0:5.1f}, T2 = {1:5.1f}, T_c = {2:5.1f}'.format(T1,T2,T_c))
fig.colorbar(cf, ax=ax1)
ax1.set_aspect('equal')
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.view_init(30, 200)
# ax2.set_aspect('equal')
surf = ax2.plot_surface(X, Y, u, 
                        cmap ='jet', lw=0, antialiased=True)